In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip uninstall -qqy jupyterlab kfp  # Remove unused conflicting packages
!pip install -qU "google-genai==1.7.0" "chromadb==0.6.3"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 54.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 73.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55

In [3]:
from google import genai
from google.genai import types

from IPython.display import Markdown

genai.__version__

'1.7.0'

In [7]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

### Data¶

Here is a small set of documents you will use to create an embedding database.

In [9]:
DOCUMENT1 = "Operating the Climate Control System  Your Googlecar has a climate control system that allows you to adjust the temperature and airflow in the car. To operate the climate control system, use the buttons and knobs located on the center console.  Temperature: The temperature knob controls the temperature inside the car. Turn the knob clockwise to increase the temperature or counterclockwise to decrease the temperature. Airflow: The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed. Mode: The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: The car will blow cool air into the car. Heat: The car will blow warm air into the car. Defrost: The car will blow warm air onto the windshield to defrost it."
DOCUMENT2 = 'Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs.'
DOCUMENT3 = "Shifting Gears Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions."

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [10]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.api_core import retry

from google.genai import types


# Define a helper to retry when per-minute quota is reached.
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})


class GeminiEmbeddingFunction(EmbeddingFunction):
    # Specify whether to generate embeddings for documents, or queries
    document_mode = True

    @retry.Retry(predicate=is_retriable)
    def __call__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        else:
            embedding_task = "retrieval_query"

        response = client.models.embed_content(
            model="models/text-embedding-004",
            contents=input,
            config=types.EmbedContentConfig(
                task_type=embedding_task,
            ),
        )
        return [e.values for e in response.embeddings]

In [12]:
from google.generativeai import embedding, configure

# Set your API key (only once)
configure(api_key=GOOGLE_API_KEY)


In [27]:
from chromadb.utils.embedding_functions import EmbeddingFunction
from google.generativeai import embedding, configure
import time
import traceback

# Only once
configure(api_key=GOOGLE_API_KEY)

class GeminiEmbeddingFunction(EmbeddingFunction):
    def __init__(self, document_mode=True, max_retries=3):
        self.document_mode = document_mode
        self.max_retries = max_retries

    def __call__(self, input: list[str]) -> list[list[float]]:
        if isinstance(input, str):
            input = [input]

        task_type = "retrieval_document" if self.document_mode else "retrieval_query"
        embeddings = []

        for text in input:
            for attempt in range(self.max_retries):
                try:
                    response = embedding.embed_content(
                        model="models/embedding-001",
                        content=text,
                        task_type=task_type,
                    )
                    embedding_result = response["embedding"]
                    embeddings.append(embedding_result)
                    break  # success, break retry loop

                except Exception as e:
                    print(f"[GeminiEmbedding] Failed on attempt {attempt+1} for input: {text[:50]}")
                    import traceback
                    traceback.print_exc()
                    time.sleep(1)
            else:
                raise RuntimeError("Failed to get embedding after retries.")

        return embeddings


### What is ChromaDB?
ChromaDB is an open-source vector database designed for use with AI/LLM applications. It helps store and search embedding vectors (like the ones you get from Gemini or OpenAI models), enabling fast and meaningful semantic search.

🧠 Why ChromaDB for LLM & RAG workflows?
Feature	Why it matters

🧩 Built for embeddings	Natively stores vectors + associated documents + metadata

⚡ Blazing fast	Uses **approximate nearest neighbor search (ANN)** for efficient similarity queries

🧪 Simple, Python-first API	Easy integration into notebooks, apps, and pipelines

🔎 Query with text	Supports semantic search: find docs similar to a query vector

🧠 LLM-native	Designed with Retrieval-Augmented Generation (RAG) in mind

🗂️ Metadata filters	You can filter results based on tags or metadata (e.g. source, date)

🧰 Portable & local	Can run in-memory, with no heavy infra required for prototyping


#### 🧪 Example Use Case with Gemini:
Convert documents to vectors using embedding-001

Store them in ChromaDB with doc IDs and metadata

When a user asks a question:

Embed the query

Use db.query(...) to find similar docs

Feed results back to Gemini for an informed answer (i.e., RAG)

## Alternatives of Chroma DB
🔄 Alternatives (Why not use...?)

FAISS	Fast, but requires more setup and doesn't store metadata as naturally

Weaviate	Powerful and scalable, but heavier for local dev

Pinecone	Managed cloud service, not free for production use

Milvus	Industrial-grade, but setup is more complex


In [29]:
embed_fn = GeminiEmbeddingFunction(document_mode=True)

db = chromadb.Client().get_or_create_collection(
    name="googlecardb",
    embedding_function=embed_fn
)

db.add(documents=documents, ids=[str(i) for i in range(len(documents))])


### Peek at the data in Chroma DB


In [38]:
db.peek(1)

{'ids': ['0'],
 'embeddings': array([[-1.83426701e-02, -2.49153264e-02, -5.98253794e-02,
          2.47534038e-03,  2.32471302e-02,  1.79663058e-02,
          4.96876426e-02, -6.08992670e-03,  3.15500982e-02,
          7.88637251e-02, -1.53274387e-02, -2.52787117e-02,
         -2.06939895e-02, -8.02255608e-03, -1.82710048e-02,
         -1.75810270e-02, -2.25012712e-02, -2.34129955e-03,
          8.86993296e-03, -4.06044051e-02, -2.04230603e-02,
         -1.21815400e-02, -3.35063711e-02, -4.04662378e-02,
          1.18617387e-02, -5.37926070e-02,  3.36588807e-02,
         -6.21514842e-02, -1.99950971e-02,  6.46993285e-03,
          4.59501054e-03,  1.62010770e-02,  2.26847478e-03,
          7.39661325e-03, -8.56751297e-03, -3.63276638e-02,
         -2.26759375e-03,  3.73643562e-02, -1.80348177e-02,
          5.04406653e-02,  1.04595860e-02, -5.33668734e-02,
         -4.47399504e-02,  7.00615393e-03, -2.48877308e-03,
          1.02346903e-02, -3.29355188e-02,  3.81240435e-02,
          2

### This returns a dictionary with the Id/Index, embeddings and metadata. The embeddings contain 768  or 769 values based on the embedding API we choose. The embedding represents the semantic meaning of the document.

### Count records in Chroma DB

In [37]:
db.count()


3

### Query Chroma DB using Natural Language

In [32]:
# Switch to query mode when generating embeddings.
embed_fn.document_mode = False

# Search the Chroma DB using the specified query.
query = "How do you use the touchscreen to play music?"

result = db.query(query_texts=[query], n_results=1)
[all_passages] = result["documents"]

Markdown(all_passages[0])

Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs.

### Prepare query with prompt that I want to add with the query

In [33]:
query_oneline = query.replace("\n", " ")

# This prompt is where you can specify any guidance on tone, or what topics the model should stick to, or avoid.
prompt = f"""You are a helpful and informative bot that answers questions using text from the reference passage included below. 
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. 
However, you are talking to a non-technical audience, so be sure to break down complicated concepts and 
strike a friendly and converstional tone. If the passage is irrelevant to the answer, you may ignore it.

QUESTION: {query_oneline}
"""

# Add the retrieved documents to the prompt.
for passage in all_passages:
    passage_oneline = passage.replace("\n", " ")
    prompt += f"PASSAGE: {passage_oneline}\n"

print(prompt)

You are a helpful and informative bot that answers questions using text from the reference passage included below. 
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. 
However, you are talking to a non-technical audience, so be sure to break down complicated concepts and 
strike a friendly and converstional tone. If the passage is irrelevant to the answer, you may ignore it.

QUESTION: How do you use the touchscreen to play music?
PASSAGE: Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs.



### Use Gen AI to generate the answer from the retrived document

In [36]:
import google.generativeai as genai
model = genai.GenerativeModel(model_name="gemini-2.0-flash-thinking-exp-1219")

answer = model.generate_content(
    contents=prompt)

Markdown(answer.text)

To play music on the touchscreen, you just need to find and touch the "Music" icon on the large touchscreen display in your Googlecar, which will then allow you to listen to your favorite songs.